In [2]:
import os

In [3]:
%pwd

'c:\\Users\\ASUS\\Documents\\FYP\\credit-card-default-prediction\\research'

In [4]:
os.chdir('..')

In [5]:
%pwd

'c:\\Users\\ASUS\\Documents\\FYP\\credit-card-default-prediction'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    scale_pos_weight: int
    max_depth: int
    min_child_weight: int
    subsample: float
    colsample_bytree: float
    objective: str
    eta: float
    gamma: float
    n_estimators: int 
    enable_categorical: str
    eval_metric: str
    alpha: float
    target_column: str

In [7]:
from default_predictor.constants import *
from default_predictor.utils.common import read_yaml, create_directories

In [21]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBoost
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            scale_pos_weight = params.scale_pos_weight,
            max_depth = params.max_depth,
            min_child_weight = params.min_child_weight,
            subsample = params.subsample,
            colsample_bytree = params.colsample_bytree,
            objective = params.objective,
            eta = params.eta,
            gamma = params.gamma,
            n_estimators = params.n_estimators ,
            enable_categorical = params.enable_categorical,
            eval_metric = params.eval_metric,
            alpha = params.alpha,
            target_column = schema.name
        )
        
        return model_trainer_config

In [22]:
import pandas as pd
import os
from default_predictor import logger
from xgboost import XGBClassifier
import joblib

In [25]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        X_train = train_data.drop([self.config.target_column],axis=1)
        X_test = test_data.drop([self.config.target_column],axis=1)
        y_train = train_data[[self.config.target_column]]
        y_test = test_data[[self.config.target_column]]
        
        params = {
        "scale_pos_weight" : self.config.scale_pos_weight,
        "max_depth" : self.config.max_depth,
        "min_child_weight" : self.config.min_child_weight,
        "subsample" : self.config.subsample,
        "colsample_bytree" : self.config.colsample_bytree,
        "objective" : self.config.objective,
        "eta" : self.config.eta,
        "gamma" : self.config.gamma,
        "n_estimators" : self.config.n_estimators ,
        "enable_categorical" : self.config.enable_categorical,
        "eval_metric" : self.config.eval_metric,
        "alpha" : self.config.alpha
        }
        
        clf = XGBClassifier(**params, random_state=42)
        clf.fit(X_train, y_train)
        
        joblib.dump(clf, os.path.join(self.config.root_dir, self.config.model_name))

In [26]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-12-12 16:24:40,761: INFO: common: YAML file config\config.yaml loaded successfuly]
[2024-12-12 16:24:40,763: INFO: common: YAML file params.yaml loaded successfuly]
[2024-12-12 16:24:40,771: INFO: common: YAML file schema.yaml loaded successfuly]
[2024-12-12 16:24:40,771: INFO: common: created directory at artifacts]
[2024-12-12 16:24:40,778: INFO: common: created directory at artifacts/model_trainer]
